# Implementation of framework
Author: Yvonne Gootzen (Statistics Netherlands & TU/e) yapm.gootzen@cbs.nl

Implementation of the metadata framework with an A* algorithm based on score functions. The notebook contains:
- Class definitions and functions (including score function), for describing the metadata problem.
- Examples of usage of the classes to illustrate the problem.
- A* implementation. 

First, import the required packages.

In [1]:
# import sys
# !{sys.executable} -m pip install networkx

## Import classes

In [2]:
from ipynb.fs.full.classes import *

## Choose only one of the two files in the below code cell!
They have different graphs, which shouldn't exist in memory at the same time, as that could lead to conflicts (variables have the same names)

In [3]:
#from ipynb.fs.full.test_cases_Yvonne_extra import *
#from ipynb.fs.full.test_cases_Yvonne import *
from ipynb.fs.full.test_case_mobility import *

## Test cases to choose from (all can be loaded in the cell above)
Tests in `test_cases_Yvonne`:
- `test_easy_Yvonne`: possible in 4 steps
- `test_impossible_Yvonne`: not possible

Tests in `test_cases_Yvonne_extra`:
- `test_medium_Yvonne`: possible in 11 steps or quicker, depending on order
- `test_combination_Yvonne`: possible in 3 steps
- `test_conversion_Yvonne`: possible in 3 steps
- `test_aggregation_Yvonne`: possible in 3 steps
- `test_context_simple`: should happen in 1 step
- `test_context`:
- `test_conversion_aggregation_Yvonne`: possible in 10 steps
- `test_hard_Yvonne`: possible in 15 - 23 steps
- `test_large_Yvonne`:
- `test_largest_Yvonne`:

Tests in `test_cases_Yvonne_extra`:
- `test_mobility`

## Choose test case (see Markdown cell above for options)
Adjust code cell below accordingly

In [4]:
test = test_mobility

Below the test objects are pulled out of the `TestCase` class so that they can be passed to `simulate`.

In [5]:
goal = test.goal
start_set = test.start_set
graphs = test.graphs
models = test.models

Make any changes to the test case (for debugging purposes).

In [6]:
#goal = Data(left_variables =[Variable("p", 0)],
#            right_variables =[Variable("r", 1),
#                              Variable("t", 2)],
#            context=["roads_in_sample", "roads_excl_sample"])
       
            
            #     context=["people_in_sample", "people_excl_sample"]) 


       #     context=["roads_in_sample", "roads_excl_sample"]) 


# A* implementation
The a_star function contains an implementation of the A* algorithm. It is currently based on the similarity score function of the set of available data sources, which is in turn based on (the sum or maximum of) the similarity score function for data sources. 

For the modelling week assignment, there are no constraints as to what can be adjusted: the similarity score function for data sources, the similarity score function for the set of sources, or even the A* implementation altogether. However, this list is currently in the suggested order of approaching the problem.

In [7]:
def get_scores(similarity_choice, open_list):
    # from all possible variants for the available set of data sources, take the one with the highest similarity score
    if similarity_choice == "sum":
        all_scores = [temp_set.similarity_sum(goal) for temp_set in open_list]
    elif similarity_choice == "max":
        all_scores = [temp_set.similarity_max(goal) for temp_set in open_list]
    elif similarity_choice == "mean":
        all_scores = [temp_set.similarity_mean(goal) for temp_set in open_list]
    elif similarity_choice == "median":
        all_scores = [temp_set.similarity_median(goal) for temp_set in open_list]
    elif similarity_choice == "min":
        all_scores = [temp_set.similarity_min(goal) for temp_set in open_list]
    elif similarity_choice == "minmax":
        all_scores = [temp_set.similarity_minmax(goal) for temp_set in open_list]
    elif similarity_choice == "maxmean":
        all_scores = [temp_set.similarity_maxmean(goal) for temp_set in open_list]
    elif similarity_choice == "maxmeanmin":
        all_scores = [temp_set.similarity_maxmeanmin(goal) for temp_set in open_list]
    elif similarity_choice == "max_per_variable":
        all_scores = [temp_set.similarity_max_per_variable(goal) for temp_set in open_list]
    elif similarity_choice == "max_per_variable_bonus":
        all_scores = [temp_set.similarity_max_per_variable_bonus(goal) for temp_set in open_list]
    else: 
        print("No known similarity score option was chosen")
        return False
    
    return all_scores

In [8]:
def prep_lookup_table(goal, start_set):
    left = copy.deepcopy(goal.left_variables)
    right = copy.deepcopy(goal.right_variables)

    start_left_variables = set()
    for d in start_set.set_of_sources:
        start_left_variables.update(d.left_variables)

    # initialize a (lookup) set of all lhs variables that can be converted to a goal lhs
    lookup_set_left = set()
    goal_left_variables_reachable = []

    for v in left:
        # for left look at conversion graph
        v_name = v.get_name()
        cg = ConversionGraph.get(v_name)
        reachable = cg.all_conversions(v.get_granularity())
        reachable = set([Variable(v_name, r) for r in reachable])    
        lookup_set_left.update(reachable)            # add it to the set

        if start_left_variables & reachable: # intersection is not empty, goal lhs variable `v` can be reached
            goal_left_variables_reachable.append(v)

    goal_left_variables_reachable = set(goal_left_variables_reachable)
    lookup_dict_right = dict()
    lookup_set_right = set()

    for v in right:
        # for right look at aggregation graph
        v_name = v.get_name()
        ag = AggregationGraph.get(v_name)
        reachable = ag.all_aggregations_reversed(v.get_granularity())
        reachable = set([Variable(v_name, r) for r in reachable])

        for r in reachable:
            key = str(r.get_name()) + str(r.get_granularity())  # key is start_set rhs variable that can reach a goal rhs variable

            if key in lookup_dict_right:
                lookup_dict_right[key].append(v) # add goal rhs variable to dict
            else:
                lookup_dict_right[key] = [v]     # add goal rhs variable to dict    

        lookup_set_right.update(reachable)       # add it to the set
        
    return lookup_set_left, lookup_set_right, goal_left_variables_reachable

In [9]:
def prep_rhs(start_set, lookup_set_right, goal_left_variables_reachable):
    start_set_copy = copy.deepcopy(start_set)
    vars_left = []   # all lhs variables in start_set with a rhs that can be aggregated to the goal rhs

    for data_source in start_set.set_of_sources:
        new_vars_right = []   # improved rhs variables

        for var_right in data_source.right_variables:
            if var_right in lookup_set_right:   # can be aggregated into some of the goal rhs variables
                goal_right_vars = lookup_dict_right[str(var_right.get_name())+str(var_right.get_granularity())]

                for v in goal_right_vars:   # for each rhs goal variable that can be reached from rhs variable of start set data source
                    new_vars_right.append(v)   # add goal rhs variable that can be reached
            else:
                new_vars_right.append(var_right) # no goal rhs variable can be reached, just add old rhs variable from start set data source
        new_vars_right = set(new_vars_right)
        new_data_source = Data(data_source.left_variables, new_vars_right, data_source.context)
        start_set_copy.add_data_source(new_data_source, "improve rhs", 0)   # add data source with (often) improved rhs

        if goal.right_variables.issubset(new_vars_right):
            for v in data_source.left_variables:
                vars_left.append(v)

    vars_left_names = set([v.get_name() for v in vars_left])

    goal_left_names = set([v.get_name() for v in goal.left_variables])

    # TODO: also check for context here!

    # for each goal lhs variable, we can get to the correct rhs from the start_set, and we can go to the correct goal lhs granularity
    if goal_left_names.issubset(vars_left_names) and goal.left_variables.issubset(goal_left_variables_reachable):                 
        print('Preprocessing succesful: goal can (modulo context, to be implemented) be reached')
        print('Starting A* algorithm')
        start = "y"
        # We don't need old start set anymore, so we delete it to keep the open list as small as possible
        start_set_copy.set_of_sources = start_set_copy.set_of_sources - start_set.set_of_sources
        agg = False # no aggregation needed anymore
    else:
        print('Preprocessing not succesful: goal cannot be reached (without modelling, to be implemented)')
        start = input("Start the A* algorithm [y/n]?")
        agg = True

    if start in ["y", "yes", "Y", "Yes"]:   # just in case user is stupid
        print('Starting A* algorithm')
    else:
        print('Terminating algorithm')
        return False
    return start_set_copy

In [39]:
# A* implementation 
    
def a_star(start_set, goal, models, max_iteration, similarity_choice = "sum", prints=False, lookUpTable = False, 
          preprocess_rhs = False, find_multiple_paths=False):
    
    # initialize open and closed lists
    open_list = []
    closed_list = []
    success_list = []
    current_set = start_set    # for printing update
    previous_score = -1
    
    # Preprocessing: look through all graphs and make a lookup table
    if lookUpTable or preprocess_rhs:
        
        lookup_set_left, lookup_set_right, goal_left_variables_reachable = prep_lookup_table(goal, start_set)
        goal.set_lookup_tables(lookup_set_left, lookup_set_right)
    
        # First make right-hand side variables of the start_set correspond to the goal, or terminate when 
        # this is not possible
        if preprocess_rhs:
            start_set_copy = prep_rhs(start_set, lookup_set_right, goal_left_variables_reachable)
            open_list.append(start_set_copy)  # add start node
    else:
        agg = True
        open_list.append(start_set)  # add start node
    
    # loop until we find the desired data set
    for i in range(max_iteration):
        if prints:
            print("--- Iteration "+str(i)+" ---")
            print("   Length open list: "+ str(len(open_list)))
            print("   Length closed list: "+ str(len(closed_list)))
           
        if i > 0:
            previous_score = all_scores[current_index]
        
        if len(open_list) == 0:
            # If the open_list is completely empty, the algorithm has failed to find a succesful path.
            
            if find_multiple_paths:
                if len(success_list) > 0:
                    return success_list
            
            return "Did not finish: open list was empty. Ran for " + str(i) + " iterations."
        
        # from all possible variants for the available set of data sources, take the one with the 
        # highest similarity score
        all_scores = get_scores(similarity_choice, open_list)
        
        # update current set
        current_index = all_scores.index(max(all_scores))  # find highest scoring index
        current_set = open_list[current_index]  # take the element with highest score
        
        # pop current set off of the open list and add it to closed list
        open_list.remove(current_set)
        closed_list.append(current_set)

        if prints:
            print("   Score of current set: " + str(all_scores[current_index]))
            #print("   Current set: \n" + str(current_set))
            print("   Current set length: " + str(len(current_set.get_sources())))
            print("   Path length: " + str(len(current_set.path)))
            #print("   Path: " + "\n         ".join(current_set.path))
            print("   Tree: " + " - ".join([str(t) for t in current_set.tree]))
            if(len(current_set.path)+len(start_set.get_sources()) -1 != len(current_set.get_sources())):
                print("   \033[1m Were some data sources removed? \033[0m")

        # check if the goal has been reached
        if(current_set.contains(goal)):
            if prints:
                print("Goal has been reached (path length: "+str(len(current_set.path))+").")
            
            if find_multiple_paths:
                success_list.append(current_set)
            else:
                return current_set
        
        #if not agg and round(previous_score,10) == round(all_scores[current_index],10):
        #    all_neighbours = current_set.get_neighbours(models=models, agg=True)
        #    print('A* is stuck, so do consider aggregation now')
        # goal not reached, search for children of the current set
        #else:
        #    all_neighbours = current_set.get_neighbours(agg=agg)
        
        # Modelling 
        # If modelling is possible, we will try this first (it is usually a good idea to prioritise this)
        n_neighbours_model = 0
        n_neighbours_nonmodel = 0
        all_neighbours_mod = current_set.get_neighbours_models(models=models)  # only modelling
        
        for neighbour in all_neighbours_mod:
            # each neighbour of the current set can be created and added to the set
            new_set_tmp = copy.deepcopy(current_set)
            new_set_tmp.add_data_source(neighbour, neighbour.path_step, i)  

            if (new_set_tmp not in open_list) and (new_set_tmp not in closed_list):
                # the new set is not already waiting to be evaluated (open_list) and has also not been 
                # evaluated yet (closed_list)
                open_list.append(new_set_tmp)
                n_neighbours_model += 1
        
        if n_neighbours_model == 0:
            # No models led to new results. So revert back to combination, aggregation and conversion
            all_neighbours_reg = current_set.get_neighbours(agg=True)  # except modelling)

            for neighbour in all_neighbours_reg:
                # each neighbour of the current set can be created and added to the set
                new_set_tmp = copy.deepcopy(current_set)
                new_set_tmp.add_data_source(neighbour, neighbour.path_step, i)  

                if (new_set_tmp not in open_list) and (new_set_tmp not in closed_list):
                    # the new set is not already waiting to be evaluated (open_list) and has also not 
                    # been evaluated yet (closed_list)
                    open_list.append(new_set_tmp)
                    n_neighbours_nonmodel += 1
                    
        print("   New neighbours: " + str(n_neighbours_model + n_neighbours_nonmodel)
             + " (model: " + str(n_neighbours_model) + ", non-model: "+str(n_neighbours_nonmodel)+")")

    if find_multiple_paths:
        return success_list
    else:
        return {"Did not finish within " + str(max_iteration) + " iterations."}
    

### Run test case (once)

In [41]:
result = a_star(start_set, 
                goal, 
                models=models,
                max_iteration=1000, 
                similarity_choice="sum", 
                prints=True, 
                lookUpTable=False, 
                preprocess_rhs=False,
                find_multiple_paths=True) 
        

--- Iteration 0 ---
   Length open list: 1
   Length closed list: 0
   Score of current set: 1.6428571428571428
   Current set length: 5
   Path length: 1
   Tree: 
   New neighbours: 5 (model: 0, non-model: 5)
--- Iteration 1 ---
   Length open list: 5
   Length closed list: 1
   Score of current set: 2.357142857142857
   Current set length: 6
   Path length: 2
   Tree: 0
   New neighbours: 4 (model: 0, non-model: 4)
--- Iteration 2 ---
   Length open list: 8
   Length closed list: 2
   Score of current set: 2.7857142857142856
   Current set length: 7
   Path length: 3
   Tree: 0 - 1
   New neighbours: 4 (model: 0, non-model: 4)
--- Iteration 3 ---
   Length open list: 11
   Length closed list: 3
   Score of current set: 3.214285714285714
   Current set length: 8
   Path length: 4
   Tree: 0 - 1 - 2
   New neighbours: 4 (model: 0, non-model: 4)
--- Iteration 4 ---
   Length open list: 14
   Length closed list: 4
   Score of current set: 3.5714285714285707
   Current set length: 9
   P

   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 52 ---
   Length open list: 17
   Length closed list: 52
   Score of current set: 4.928571428571429
   Current set length: 14
   Path length: 10
   Tree: 0 - 1 - 2 - 3 - 4 - 5 - 32 - 34 - 35
   New neighbours: 2 (model: 0, non-model: 2)
--- Iteration 53 ---
   Length open list: 18
   Length closed list: 53
   Score of current set: 5.357142857142858
   Current set length: 15
   Path length: 11
   Tree: 0 - 1 - 2 - 3 - 4 - 5 - 32 - 34 - 35 - 52
   New neighbours: 1 (model: 0, non-model: 1)
--- Iteration 54 ---
   Length open list: 18
   Length closed list: 54
   Score of current set: 5.357142857142858
   Current set length: 16
   Path length: 12
   Tree: 0 - 1 - 2 - 3 - 4 - 5 - 32 - 34 - 35 - 52 - 53
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 55 ---
   Length open list: 17
   Length closed list: 55
   Score of current set: 4.928571428571429
   Current set length: 15
   Path length: 11
   Tree: 0 - 1 - 2 - 3 - 4 

   Score of current set: 6.071428571428572
   Current set length: 17
   Path length: 13
   Tree: 0 - 1 - 2 - 3 - 73 - 74 - 75 - 76 - 77 - 78 - 88 - 89
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 91 ---
   Length open list: 18
   Length closed list: 91
   Score of current set: 5.642857142857143
   Current set length: 16
   Path length: 12
   Tree: 0 - 1 - 2 - 3 - 73 - 74 - 75 - 76 - 77 - 78 - 88
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 92 ---
   Length open list: 17
   Length closed list: 92
   Score of current set: 5.285714285714286
   Current set length: 15
   Path length: 11
   Tree: 0 - 1 - 2 - 3 - 73 - 74 - 75 - 76 - 77 - 78
   New neighbours: 1 (model: 0, non-model: 1)
--- Iteration 93 ---
   Length open list: 17
   Length closed list: 93
   Score of current set: 5.714285714285714
   Current set length: 16
   Path length: 12
   Tree: 0 - 1 - 2 - 3 - 73 - 74 - 75 - 76 - 77 - 78 - 92
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 94 --

   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 130 ---
   Length open list: 17
   Length closed list: 130
   Score of current set: 6.357142857142858
   Current set length: 16
   Path length: 12
   Tree: 0 - 1 - 2 - 3 - 73 - 101 - 119 - 120 - 121 - 122 - 123
Goal has been reached (path length: 12).
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 131 ---
   Length open list: 16
   Length closed list: 131
   Score of current set: 5.285714285714286
   Current set length: 14
   Path length: 10
   Tree: 0 - 1 - 2 - 3 - 73 - 101 - 119 - 120 - 121
   New neighbours: 2 (model: 0, non-model: 2)
--- Iteration 132 ---
   Length open list: 17
   Length closed list: 132
   Score of current set: 5.714285714285714
   Current set length: 15
   Path length: 11
   Tree: 0 - 1 - 2 - 3 - 73 - 101 - 119 - 120 - 121 - 131
   New neighbours: 1 (model: 0, non-model: 1)
--- Iteration 133 ---
   Length open list: 17
   Length closed list: 133
   Score of current set: 5.714285714285714
  

   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 168 ---
   Length open list: 14
   Length closed list: 168
   Score of current set: 6.357142857142858
   Current set length: 15
   Path length: 11
   Tree: 0 - 1 - 2 - 3 - 158 - 159 - 160 - 161 - 162 - 163
Goal has been reached (path length: 11).
   New neighbours: 1 (model: 0, non-model: 1)
--- Iteration 169 ---
   Length open list: 14
   Length closed list: 169
   Score of current set: 6.357142857142858
   Current set length: 16
   Path length: 12
   Tree: 0 - 1 - 2 - 3 - 158 - 159 - 160 - 161 - 162 - 163 - 168
Goal has been reached (path length: 12).
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 170 ---
   Length open list: 13
   Length closed list: 170
   Score of current set: 6.0
   Current set length: 15
   Path length: 11
   Tree: 0 - 1 - 2 - 3 - 158 - 159 - 160 - 161 - 162 - 163
Goal has been reached (path length: 11).
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 171 ---
   Length open list

...

   New neighbours: 1 (model: 0, non-model: 1)
--- Iteration 682 ---
   Length open list: 9
   Length closed list: 682
   Score of current set: 4.214285714285714
   Current set length: 14
   Path length: 10
   Tree: 0 - 446 - 665 - 666 - 667 - 668 - 669 - 680 - 681
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 683 ---
   Length open list: 8
   Length closed list: 683
   Score of current set: 3.7857142857142856
   Current set length: 13
   Path length: 9
   Tree: 0 - 446 - 665 - 666 - 667 - 668 - 669 - 680
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 684 ---
   Length open list: 7
   Length closed list: 684
   Score of current set: 3.4285714285714284
   Current set length: 12
   Path length: 8
   Tree: 0 - 446 - 665 - 666 - 667 - 668 - 669
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 685 ---
   Length open list: 6
   Length closed list: 685
   Score of current set: 2.4285714285714284
   Current set length: 8
   Path length: 4
   Tree: 0 - 446 -

   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 723 ---
   Length open list: 10
   Length closed list: 723
   Score of current set: 4.142857142857143
   Current set length: 14
   Path length: 10
   Tree: 0 - 697 - 698 - 699 - 700 - 701 - 702 - 703 - 720
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 724 ---
   Length open list: 9
   Length closed list: 724
   Score of current set: 3.7857142857142856
   Current set length: 13
   Path length: 9
   Tree: 0 - 697 - 698 - 699 - 700 - 701 - 702 - 703
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 725 ---
   Length open list: 8
   Length closed list: 725
   Score of current set: 2.7857142857142856
   Current set length: 8
   Path length: 4
   Tree: 0 - 697 - 698
   New neighbours: 1 (model: 0, non-model: 1)
--- Iteration 726 ---
   Length open list: 8
   Length closed list: 726
   Score of current set: 2.7857142857142856
   Current set length: 9
   Path length: 5
   Tree: 0 - 697 - 698 - 699
   New neighb

   Score of current set: 6.5
   Current set length: 16
   Path length: 12
   Tree: 0 - 697 - 698 - 768 - 769 - 770 - 771 - 772 - 773 - 774 - 775
Goal has been reached (path length: 12).
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 777 ---
   Length open list: 10
   Length closed list: 777
   Score of current set: 6.142857142857143
   Current set length: 15
   Path length: 11
   Tree: 0 - 697 - 698 - 768 - 769 - 770 - 771 - 772 - 773 - 774
Goal has been reached (path length: 11).
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 778 ---
   Length open list: 9
   Length closed list: 778
   Score of current set: 5.571428571428571
   Current set length: 14
   Path length: 10
   Tree: 0 - 697 - 698 - 768 - 769 - 770 - 771 - 772 - 773
Goal has been reached (path length: 10).
   New neighbours: 1 (model: 0, non-model: 1)
--- Iteration 779 ---
   Length open list: 9
   Length closed list: 779
   Score of current set: 5.571428571428571
   Current set length: 15
   Path 

   New neighbours: 1 (model: 0, non-model: 1)
--- Iteration 830 ---
   Length open list: 4
   Length closed list: 830
   Score of current set: 4.214285714285714
   Current set length: 13
   Path length: 9
   Tree: 0 - 697 - 809 - 816 - 817 - 818 - 828 - 829
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 831 ---
   Length open list: 3
   Length closed list: 831
   Score of current set: 2.714285714285714
   Current set length: 10
   Path length: 6
   Tree: 0 - 697 - 809 - 816 - 817
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 832 ---
   Length open list: 2
   Length closed list: 832
   Score of current set: 2.0
   Current set length: 6
   Path length: 2
   Tree: 0
   New neighbours: 3 (model: 0, non-model: 3)
--- Iteration 833 ---
   Length open list: 4
   Length closed list: 833
   Score of current set: 2.357142857142857
   Current set length: 7
   Path length: 3
   Tree: 0 - 832
   New neighbours: 2 (model: 0, non-model: 2)
--- Iteration 834 ---
   Length op

   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 883 ---
   Length open list: 2
   Length closed list: 883
   Score of current set: 2.642857142857143
   Current set length: 10
   Path length: 6
   Tree: 0 - 832 - 878 - 879 - 880
   New neighbours: 0 (model: 0, non-model: 0)
--- Iteration 884 ---
   Length open list: 1
   Length closed list: 884
   Score of current set: 1.6428571428571428
   Current set length: 6
   Path length: 2
   Tree: 0
   New neighbours: 1 (model: 1, non-model: 0)
--- Iteration 885 ---
   Length open list: 1
   Length closed list: 885
   Score of current set: 1.6428571428571428
   Current set length: 7
   Path length: 3
   Tree: 0 - 884
   New neighbours: 1 (model: 1, non-model: 0)
--- Iteration 886 ---
   Length open list: 1
   Length closed list: 886
   Score of current set: 2.2857142857142856
   Current set length: 8
   Path length: 4
   Tree: 0 - 884 - 885
   New neighbours: 2 (model: 0, non-model: 2)
--- Iteration 887 ---
   Length open list: 2
   

In [43]:
len(result)

264

### Find quickest path
If the option find_multiple_paths=True was used, there may now be multiple paths in the result of the A* algorithm. Later, quality assignments will help with choosing a specific path. For now, we are interested in the shortest path.

In [47]:
path_lengths = [len(x.path) for x in result]
shortest_path_index = path_lengths.index(min(path_lengths))  # find shortest path index
shortest_path = result[shortest_path_index]  # take the path with the shortest path
print("Path: " + "\n      ".join(shortest_path.path))
            

Path: start_set
      aggregate (t0 to t2): (c0 | r0, t2)_['roads_in_sample']
      aggregate (t1 to t2): (m0 | p0, t2)_['people_in_sample']
      model (Modality Choice model):(p0 | od0, t2)_['people_excl_sample', 'people_in_sample']
      model (Shortest Path model):(p0 | r1, t2)_['roads_excl_sample', 'roads_in_sample']
      aggregate (r1 to r0): (p0 | r0, t2)_['roads_excl_sample', 'roads_in_sample']
      combine (columnwise): (c0, p0 | r0, t2)_['roads_in_sample']
      model (Calibration model):(c0 | r0)_['roads_excl_sample', 'roads_in_sample']


### Run test case (multiple times)

In [40]:
def simulate(n_simulations, start_set, goal, max_iteration, similarity_choice = "sum", 
             prints = False, similarity_variant = "normalized_basic", lookUpTable = False, preprocess_rhs = False):
    """Do multiple a* algorithms to get an average mean score"""
    goal.similarity_variant = similarity_variant  # similarity score computation per data source
    
    t = time.perf_counter()
    times = np.zeros(n_simulations)
    t_start = t
    
    for k in range(n_simulations):
        result = a_star(start_set, goal, max_iteration, similarity_choice, prints, lookUpTable, preprocess_rhs) 
        # check if we did not finish
        if 'Did not finish' in result.values():
            print(f"Couldn't finish one of the simulations in {max_iteration} iterations.")
            return
        
        times[k] = time.perf_counter() - t_start
        t_start = time.perf_counter()
    # if we got through the whole loop
    t_avg = round(np.mean(times),5)
    CI_half_length = round(1.96 * np.std(times, ddof=1) / np.sqrt(n_simulations),10) # ddof=1 to get sample standard deviation (normalize by N-1)
    print(f"Average time to simulate: {t_avg} ± {CI_half_length}.")

In [14]:
#simulate(1, start_set, 
#       goal,
#       max_iteration=100, 
#       similarity_choice = "max_per_variable_bonus", # max, sum, var
#       prints=True, 
#       similarity_variant = "normalized_basic",
#       lookUpTable = True,
#       preprocess_rhs = True) #True)